In [1]:
import cv2
import depthai as dai
import numpy as np

import time
from pathlib import Path
from matplotlib import pyplot as plt

In [2]:
img1 = cv2.imread('video_frames/17137219437727.png')
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

img2 = cv2.imread('video_frames/17137219432663.png')
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

In [3]:
x1, y1 = 100, 100
x2, y2 = 150, 150
patch_size = 5

patch_img1 = img1[y1-patch_size//2:y1+patch_size//2+1, x1-patch_size//2:x1+patch_size//2+1]
patch_img2 = img2[y2-patch_size//2:y2+patch_size//2+1, x2-patch_size//2:x2+patch_size//2+1]

# Display the selected patches
cv2.imshow('Patch in Image 1', patch_img1)
cv2.imshow('Patch in Image 2', patch_img2)
cv2.waitKey(1000)
cv2.destroyAllWindows()


In [4]:
# Computing the SIFT feature for each of these 2 patches

sift = cv2.SIFT_create() 

kp_img1, desc_img1 = sift.detectAndCompute(img1, None) 
kp_img2, desc_img2 = sift.detectAndCompute(img2, None) 

bf = cv2.BFMatcher()
matches = bf.knnMatch(desc_img1, desc_img2, k=2)

good_points=[]     
for m, n in matches: 
    if(m.distance < 0.6*n.distance): 
        good_points.append(m) 

In [5]:
# Filter descriptors based on matches
matched_desc_img1 = np.array([desc_img1[m.queryIdx] for m in good_points])
matched_desc_img2 = np.array([desc_img2[m.trainIdx] for m in good_points])

# Compute the Sum of Squared Difference (SSD) between SIFT descriptors
ssd_value = np.sum((matched_desc_img1 - matched_desc_img2) ** 2)
print("SSD value:", ssd_value)


SSD value: 11045979.0


In [6]:
query_pts = np.float32([kp_img1[m.queryIdx] 
                .pt for m in good_points]).reshape(-1, 1, 2) 
 
train_pts = np.float32([kp_img2[m.trainIdx] 
                .pt for m in good_points]).reshape(-1, 1, 2) 
 
matrix, mask = cv2.findHomography(query_pts, train_pts, cv2.RANSAC, 5.0) 
 
matches_mask = mask.ravel().tolist() 

h,w = img1.shape
 
pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
 
dst = cv2.perspectiveTransform(pts, matrix)
 
homography = cv2.polylines(img2, [np.int32(dst)], True, (255, 0, 0), 3) 
 
cv2.imshow("Homography", homography) 
cv2.imshow("Img", img1) 
cv2.waitKey(1000)
cv2.destroyAllWindows()

In [7]:
# Homography matrix
matrix

array([[ 9.73705303e-01,  2.24858628e-02,  1.80161356e+01],
       [-5.66427120e-03,  9.96127926e-01, -1.63754839e+00],
       [-5.48952861e-05,  3.99859072e-05,  1.00000000e+00]])

In [8]:
img3 = cv2.drawMatches(img1, kp_img1, img2, kp_img2, good_points, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.imshow("lines", img3)
cv2.waitKey(1000)
cv2.imwrite("homography.png", img3)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
# Compute the inverse of the homography matrix
inverse_matrix = np.linalg.inv(matrix)
print("Inverse Homography Matrix:")
print(inverse_matrix)

Inverse Homography Matrix:
[[ 1.02583038e+00 -2.24129990e-02 -1.85182016e+01]
 [ 5.92535260e-03  1.00369168e+00  1.53684174e+00]
 [ 5.60763219e-05 -4.13638904e-05  9.98921986e-01]]
